In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# If memory growth is enabled for a PhysicalDevice,
# the runtime initialization will not allocate all memory on the device. 
# Memory growth cannot be configured on a PhysicalDevice with virtual devices configured.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

---

In [2]:
import sys
sys.path.append("./backend/models/src/")
from munge import prepare_historical_data

In [4]:
df = prepare_historical_data(5, ["Longitude", "Latitude", "Magnitude"])

In [5]:
df

,magnitude,geohash,geohash_idx
0,6.0,x5,385
1,5.8,wb,358
2,6.2,2h,17
3,5.8,5m,50
4,5.8,wf,362
...,...,...,...
23407,5.6,9q,99
23408,5.5,9q,99
23409,5.9,xn,391
23410,6.3,qw,260


---

In [34]:
import training as train

In [38]:
y = train.get_y(df)
y

array([[385],
       [358],
       [ 17],
       ...,
       [391],
       [260],
       [391]], dtype=int16)

In [ ]:
# for plotting... (x axis)
# date_train = time_range[:split_idx]
# date_test = time_range[split_idx:]
# len(date_train), len(date_test)

In [16]:
# need to choose batch size so that n/batch_size remainder is memory (timesteps/events looking back)
# want larger batch size for model performance so wrote wrapper func to help find


# train/test split
for train_proportion in list(range(70, 90, 1)):
    train_proportion /= 100
    split_idx = int(train_proportion*len(y))
    geo_train = y[:split_idx]
    geo_test = y[split_idx:]


    for MEMORY in list(range(1, 10)):
        def print_factors(data_ct, MEMORY):
            return {batch_size for batch_size in range(1, data_ct+1) 
                        if data_ct % batch_size == MEMORY}

        train_batch_sizes = print_factors(len(geo_train), MEMORY)
        test_batch_sizes = print_factors(len(geo_test), MEMORY)


        common_sizes = train_batch_sizes.intersection(test_batch_sizes)
        if len(common_sizes)>=1:
            print(f"TT SPLIT: {train_proportion} | MEMORY: {MEMORY} | COMMON BATCH SIZES: {common_sizes}")

TT SPLIT: 0.7 | MEMORY: 1 | COMMON BATCH SIZES: {2341}
TT SPLIT: 0.71 | MEMORY: 2 | COMMON BATCH SIZES: {4}
TT SPLIT: 0.71 | MEMORY: 6 | COMMON BATCH SIZES: {8}
TT SPLIT: 0.72 | MEMORY: 1 | COMMON BATCH SIZES: {5}
TT SPLIT: 0.72 | MEMORY: 6 | COMMON BATCH SIZES: {25, 10, 50}
TT SPLIT: 0.73 | MEMORY: 2 | COMMON BATCH SIZES: {8, 16, 4}
TT SPLIT: 0.75 | MEMORY: 1 | COMMON BATCH SIZES: {2}
TT SPLIT: 0.75 | MEMORY: 3 | COMMON BATCH SIZES: {6}
TT SPLIT: 0.76 | MEMORY: 1 | COMMON BATCH SIZES: {2}
TT SPLIT: 0.76 | MEMORY: 3 | COMMON BATCH SIZES: {6}
TT SPLIT: 0.77 | MEMORY: 1 | COMMON BATCH SIZES: {2}
TT SPLIT: 0.77 | MEMORY: 2 | COMMON BATCH SIZES: {7}
TT SPLIT: 0.77 | MEMORY: 3 | COMMON BATCH SIZES: {6}
TT SPLIT: 0.77 | MEMORY: 9 | COMMON BATCH SIZES: {42, 21, 14}
TT SPLIT: 0.78 | MEMORY: 1 | COMMON BATCH SIZES: {2, 10, 5}
TT SPLIT: 0.78 | MEMORY: 2 | COMMON BATCH SIZES: {19}
TT SPLIT: 0.78 | MEMORY: 3 | COMMON BATCH SIZES: {6}
TT SPLIT: 0.78 | MEMORY: 6 | COMMON BATCH SIZES: {15}
TT SPLIT: 

In [ ]:
tt_split(y, train_proportion=.77)

In [18]:
MEMORY = 9
BATCH_SIZE = 42

train_generator = TimeseriesGenerator(geo_train, geo_train, length=MEMORY, batch_size=BATCH_SIZE)

In [25]:
DROPOUT = .6
EPOCHS = 10
ACTIVATION = "tanh"
ALPHA = 1e-06
UNITS = 64
CLASS_COUNT = len(np.unique(y))
feature_ct = geo_train.shape[1]

In [26]:
model = Sequential()

# did not 1-hot encode since using  sparse loss
batchsize__input_dim = (BATCH_SIZE, MEMORY, feature_ct) 
model.add(LSTM(UNITS, 
               activation=ACTIVATION, 
               batch_input_shape=batchsize__input_dim, 
               return_sequences=True, 
               stateful=True, 
               name="input_lstm",
               recurrent_dropout=DROPOUT
              )) 

model.add(LSTM(UNITS, 
               activation=ACTIVATION,
               recurrent_dropout=DROPOUT,
               return_sequences=False,
              ))

model.add(Dropout(DROPOUT))

model.add(Dense(CLASS_COUNT, activation="softmax"))


def top_k_metric(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=10) 


optimizer = Adam(learning_rate=ALPHA)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, 
              sample_weight_mode="temporal",
              metrics=[top_k_metric, "sparse_categorical_accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_lstm (LSTM)            (42, 9, 64)               16896     
_________________________________________________________________
lstm_1 (LSTM)                (42, 64)                  33024     
_________________________________________________________________
dropout_1 (Dropout)          (42, 64)                  0         
_________________________________________________________________
dense (Dense)                (42, 424)                 27560     
Total params: 77,480
Trainable params: 77,480
Non-trainable params: 0
_________________________________________________________________


In [27]:
for i in range(EPOCHS):
    print()
    
    model.fit_generator(train_generator, epochs=1, shuffle=False)
    model.reset_states()

2021-10-19 20:30:52.637147: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-19 20:30:52.670645: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3494665000 Hz
2021-10-19 20:31:00.944460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


429/429 [==============================] - 30s 45ms/step - loss: 6.0595 - top_k_metric: 0.0161 - sparse_categorical_accuracy: 0.0026

429/429 [==============================] - 11s 27ms/step - loss: 6.0530 - top_k_metric: 0.0154 - sparse_categorical_accuracy: 0.0027

429/429 [==============================] - 11s 26ms/step - loss: 6.0477 - top_k_metric: 0.0164 - sparse_categorical_accuracy: 0.0023

429/429 [==============================] - 11s 25ms/step - loss: 6.0411 - top_k_metric: 0.0152 - sparse_categorical_accuracy: 0.0023

429/429 [==============================] - 11s 26ms/step - loss: 6.0387 - top_k_metric: 0.0138 - sparse_categorical_accuracy: 0.0029

429/429 [==============================] - 11s 26ms/step - loss: 6.0311 - top_k_metric: 0.0133 - sparse_categorical_accuracy: 0.0042

429/429 [==============================] - 11s 25ms/step - loss: 6.0260 - top_k_metric: 0.0137 - sparse_categorical_accuracy: 0.0043

429/429 [==============================] - 12s 29ms/step - los

One big difference between regular regression models and time series models is how we run predictions. The first one should be pretty obvious, we take the last 12 months of train data and predict it to get the first test data.
How do we predict the next one?
This is a big issue esp. if you take a shortcut and use the test data’s first value and use that as your last prediction. That way you are feeding the correct values for the prior steps helping the model to create better results that it would otherwise give.
What needs to happen is that the “first prediction” needs to be added to the last 11 training data to create a new set of 12 data points to predict the next one. This way we are not cheating at all, the test data is really test data and is never seen by the model.

this post wasn't worth saving tho...

In [ ]:
# trimming slightly to fit batch size
test_generator = TimeseriesGenerator(geo_test, geo_test, length=MEMORY, batch_size=BATCH_SIZE)
prediction = model.predict_generator(test_generator, verbose=1)
prediction.shape

In [ ]:
max_index_col = np.argmax(prediction, axis=1)
set(max_index_col) # the different indices that were max (output neurons that lit up the most)

In [ ]:
lat__long = [gh.decode(geohash_idx2hash[e]) for e in max_index_col] # make sure right way around!
# make sure proper order
# make sure unpacks index per model lines up. MOVE ON IGNORE DEETS!
prediction_df = pd.DataFrame(lat__long, columns=["latitude", "longitude"])
prediction_df.drop_duplicates(inplace=True)
prediction_df.reset_index(drop=True)
prediction_df

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=prediction_df['longitude'], y=prediction_df['latitude'], color="red")

huh = df[["Latitude", "Longitude"]].drop_duplicates()
plt.scatter(x=huh['Longitude'], y=huh['Latitude'], color="green", alpha=.01)
plt.show()

Assuming that the algorithms have a hard time to interpret GPS coordinates, the system
could instead use encoded positions. This approach would take a latitude and longitude position and with a defined precision hash it and save it in a dictionary. This precision would
be the amount of decimals for the latitude and longitude to use. This would discretize the
operating area into a grid where the interval depends on the chosen precision.
The drawback of this approach is with higher precision and operating area, the exponentially more locations have to hashed and stored. A small area of radius ten kilometre and
4 decimal precision could yield well over 300000 locations and therefore outcomes. Another
drawback is that the algorithm can only use positions it have trained on.


need to 1-hjot encode since no ordinal relation exists

In [18]:
import sys
sys.path.append("./backend/src/")
import utils, json

In [22]:
with open("./backend/src/dev/test.json", "r") as infile:
    data = json.load(infile)
df = utils.frame_data([utils.flatten_json(q) for q in data["features"]], keep2rename = {'properties>mag':'magnitude',
               'properties>sig':'significance',
               'geometry>coordinates>0':'longitude',
               'geometry>coordinates>1':'latitude',
               'geometry>coordinates>2':'depth'}
)

In [24]:
df["magnitude"].unique()

array([0.78, 0.86, 2.43, 1.9 , 1.84, 0.98, 1.18, 0.94, 2.88, 0.84, 2.19,
       1.93, 1.  ])